In [1]:
import sys
import numpy as np
import tensorflow as tf
import os
from tensorflow.contrib.lookup import MutableHashTable
from tensor2tensor.layers import common_layers

from tensor2tensor.models import transformer
sys.path.append('/workspace/MT/tensor2tensor/tensor2tensor/models/')

from transformer_test import TransformerTest
from transformer_cache import TransformerCache
from tensor2tensor.data_generators import problem_hparams
from transformer_cache import LRUCache
from tensor2tensor.data_generators import problem

sys.path.append('/workspace/MT/t2t_data_generators/')

from generator import ShadENRUOpusProblem

In [2]:
shad_problem = ShadENRUOpusProblem()

data = shad_problem.generate_encoded_samples(
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    problem.DatasetSplit.TEST
)

INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


[2018-03-20 09:50:23,611] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


[2018-03-20 09:50:23,709] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


In [3]:
has_input = True
mode=tf.estimator.ModeKeys.EVAL

hparams = transformer.transformer_base_single_gpu()
hparams.data_dir =  '/workspace/MT/shad_nlp18_contextNMT/data_4prev//'
p_hparams = shad_problem.get_hparams(hparams)

## Initializing the model

In [4]:
model = TransformerCache(hparams, mode, p_hparams)

[('batch_size_multiplier', 1), ('input_modality', {'inputs': ('symbol', 22057)}), ('input_space_id', 0), ('loss_multiplier', 1.0), ('stop_at_eos', 1), ('target_modality', ('symbol', 23180)), ('target_space_id', 0), ('vocabulary', {'inputs': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f0b138a3cc0>, 'targets': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7f0b138a3d68>}), ('was_copy', False), ('was_reversed', False)]
INFO:tensorflow:Unsetting shared_embedding_and_softmax_weights.


[2018-03-20 09:50:24,079] Unsetting shared_embedding_and_softmax_weights.


INFO:tensorflow:Setting T2TModel mode to 'eval'


[2018-03-20 09:50:24,081] Setting T2TModel mode to 'eval'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

[2018-03-20 09:50:24,115] Setting hparams.symbol_dropout to 0.0


INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0


[2018-03-20 09:50:24,116] Setting hparams.layer_prepostprocess_dropout to 0.0


INFO:tensorflow:Setting hparams.attention_dropout to 0.0


[2018-03-20 09:50:24,117] Setting hparams.attention_dropout to 0.0


In [5]:
features_true = {
    "inputs": tf.placeholder(shape=[None, None, 1, 1], dtype=tf.int32, name="inputs"),
    "targets": tf.placeholder(shape=[None, None, 1, 1], dtype=tf.int32, name="inputs"),
    "target_space_id": tf.constant(1, dtype=tf.int32)
}

In [6]:
data = shad_problem.generate_encoded_samples(
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    '/workspace/MT/shad_nlp18_contextNMT/data_4prev/',
    problem.DatasetSplit.TEST
)

INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


[2018-03-20 09:50:24,150] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.src


INFO:tensorflow:Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


[2018-03-20 09:50:24,242] Found vocab file: /workspace/MT/shad_nlp18_contextNMT/data_4prev/vocab_enru.dst


In [7]:
decode_length = tf.placeholder(dtype=tf.int32)

In [8]:
fast_result = model._greedy_infer(features_true, decode_length)["outputs"]

## Loading the weights

In [9]:
save_dict = {"/".join(["transformer"] + var.name[:-2].split('/')[1:]) : var for var in tf.global_variables()[3:]}

In [10]:
saver = tf.train.Saver(save_dict)
ckpt = tf.train.get_checkpoint_state('/workspace/MT/train/transformer/')

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [12]:
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("...no checkpoint found...")

INFO:tensorflow:Restoring parameters from /workspace/MT/train/transformer/model.ckpt-250000


[2018-03-20 09:50:33,351] Restoring parameters from /workspace/MT/train/transformer/model.ckpt-250000


In [13]:
new_saver = tf.train.Saver(tf.global_variables()[:2])
new_ckpt = tf.train.get_checkpoint_state('/workspace/MT/train_cache/')

In [14]:
if new_ckpt and new_ckpt.model_checkpoint_path:
    new_saver.restore(sess, new_ckpt.model_checkpoint_path)
else:
    print("...no checkpoint found...")

INFO:tensorflow:Restoring parameters from /workspace/MT/train_cache/transformer_cache


[2018-03-20 09:50:33,832] Restoring parameters from /workspace/MT/train_cache/transformer_cache


In [15]:
encoders = shad_problem.feature_encoders('/workspace/MT/shad_nlp18_contextNMT/data_4prev/')

## Predicting

In [16]:
counter = 0
flusher = 0
with open('/workspace/MT/transformer_cache_test_ru.dst', 'w', encoding='utf8') as file:
    for it in data:
        inputs = np.array(it['inputs']).reshape((1, len(it['inputs']), 1, 1))
        targets = np.array(it['targets']).reshape((1, len(it['targets']), 1, 1))
        kek = sess.run([fast_result], feed_dict={
            features_true['inputs'] : inputs,
            features_true['targets'] : targets,
            decode_length : len(targets)
        })
        string = encoders['targets'].decode(np.squeeze(kek[0])[:-1])
        print(string, file=file)
        counter += 1
        flusher += 1
        if flusher == 5:
            model.sentence_cache.Flush()
            flusher = 0
        if counter == 5000:
            break

In [17]:
counter = 0
with open('/workspace/MT/shad_nlp18_contextNMT/data_fused/en_test.src', 'r', encoding='utf8') as file:
    with open('/workspace/MT/shad_nlp18_contextNMT/data_fused/en_test_short.src', 'w', encoding='utf8') as wfile:
        for line in file:
            print(line, file=wfile)
            counter += 1
            if counter == 5000:
                break

In [8]:
counter = 0
with open('/workspace/MT/shad_nlp18_contextNMT/data_fused/ru_test.dst', 'r', encoding='utf8') as file:
    with open('/workspace/MT/shad_nlp18_contextNMT/data_fused/ru_test_short.dst', 'w', encoding='utf8') as wfile:
        for line in file:
            print(line, end='', file=wfile)
            counter += 1
            if counter == 5000:
                break